# Exp 5. Target Encoding 

In [1]:
# Import Modules
import os
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
# Enable plotly offline plotting
import plotly
import plotly.graph_objects as go
import seaborn as sns
from IPython.core.display import HTML, display
from ipywidgets import interact


# Ignore sklearn warnings
def warn(*args, **kwargs):
    pass


warnings.warn = warn


plotly.offline.init_notebook_mode(connected=True)

# Init project path
PROJECT_DIR = os.getcwd() + "/../../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2

display(HTML("<style>.container { width:100% !important; }</style>"))

In [123]:
use_cols = [
    "target",
    "target_address_1_mean",
    "target_address_1_2_mean",
    "target_station_1_mean",
    "target_station_2_mean",
    "target_devided_by_area_address_1_mean",
    "target_devided_by_area_address_1_2_mean",
    "target_devided_by_area_station_1_mean",
    "target_devided_by_area_station_2_mean",
    "target_address_1_area",
    "target_address_1_2_area",
    "target_station_1_area",
    "target_station_2_area",
    "duration_1",
    "ohe_bathroom_専用バス",
    "ohe_bathroom_専用トイレ",
    "ohe_bathroom_バス・トイレ別",
    "ohe_bathroom_シャワー",
    "ohe_bathroom_追焚機能",
    "ohe_bathroom_温水洗浄便座",
    "ohe_bathroom_洗面台独立",
    "ohe_bathroom_脱衣所",
    "ohe_bathroom_浴室乾燥機",
    "ohe_bathroom_バスなし",
    "ohe_bathroom_共同バス",
    "ohe_bathroom_共同トイレ",
    "ohe_equipment_エアコン付",
    "ohe_equipment_シューズボックス",
    "ohe_equipment_バルコニー",
    "ohe_equipment_フローリング",
    "ohe_equipment_エレベーター",
    "ohe_equipment_公営水道",
    "ohe_equipment_下水",
    "ohe_equipment_都市ガス",
    "ohe_equipment_室内洗濯機置場",
    "ohe_equipment_2面採光",
    "ohe_equipment_タイル張り",
    "ohe_equipment_室外洗濯機置場",
    "ohe_equipment_冷房",
    "ohe_equipment_敷地内ごみ置き場",
    "ohe_equipment_ロフト付き",
    "ohe_equipment_3面採光",
    "ohe_equipment_24時間換気システム",
    "ohe_equipment_水道その他",
    "ohe_equipment_プロパンガス",
    "ohe_equipment_ウォークインクローゼット",
    "ohe_equipment_トランクルーム",
    "ohe_equipment_出窓",
    "ohe_equipment_ペアガラス",
    "ohe_equipment_専用庭",
    "ohe_equipment_バリアフリー",
    "ohe_equipment_床暖房",
    "ohe_equipment_床下収納",
    "ohe_equipment_クッションフロア",
    "ohe_equipment_オール電化",
    "ohe_equipment_二重サッシ",
    "ohe_equipment_ルーフバルコニー",
    #     "ohe_equipment_ガス暖房",
    #     "ohe_equipment_ガスその他",
    "ohe_equipment_洗濯機置場なし",
    "ohe_equipment_防音室",
    #     "ohe_equipment_排水その他",
    #     "ohe_equipment_地下室",
    #     "ohe_equipment_浄化槽",
    #     "ohe_equipment_汲み取り",
    #     "ohe_equipment_石油暖房",
    #     "ohe_equipment_井戸",
    #     "ohe_equipment_二世帯住宅",
    "ohe_internet_インターネット対応",
    "ohe_internet_CSアンテナ",
    "ohe_internet_BSアンテナ",
    "ohe_internet_インターネット使用料無料",
    "ohe_internet_光ファイバー",
    "ohe_internet_CATV",
    "ohe_internet_高速インターネット",
    "ohe_internet_有線放送",
    "ohe_kitchen_ガスコンロ",
    "ohe_kitchen_コンロ3口",
    "ohe_kitchen_システムキッチン",
    "ohe_kitchen_給湯",
    "ohe_kitchen_L字キッチン",
    "ohe_kitchen_コンロ2口",
    "ohe_kitchen_コンロ設置可（口数不明）",
    "ohe_kitchen_コンロ1口",
    "ohe_kitchen_カウンターキッチン",
    "ohe_kitchen_独立キッチン",
    "ohe_kitchen_IHコンロ",
    "ohe_kitchen_冷蔵庫あり",
    "ohe_kitchen_コンロ設置可（コンロ1口）",
    "ohe_kitchen_コンロ設置可（コンロ2口）",
    "ohe_kitchen_電気コンロ",
    "ohe_kitchen_コンロ設置可（コンロ3口）",
    "ohe_kitchen_コンロ4口以上",
    #     "ohe_kitchen_コンロ設置可（コンロ4口以上）",
    "ohe_parking_駐車場",
    "ohe_parking_駐輪場",
    "ohe_direction_北",
    "ohe_direction_北東",
    "ohe_direction_北西",
    "ohe_direction_南",
    "ohe_direction_南東",
    "ohe_direction_南西",
    "ohe_direction_東",
    "ohe_direction_西",
    "ohe_layout_1DK",
    "ohe_layout_1K",
    "ohe_layout_1LDK",
    "ohe_layout_2DK",
    "ohe_layout_2K",
    "ohe_layout_2LDK",
    "ohe_layout_3DK",
    "ohe_layout_3K",
    "ohe_layout_3LDK",
    #     "ohe_layout_4DK",
    #     "ohe_layout_4LDK",
    #     "ohe_layout_5DK",
    "ohe_layout_4K以上",
    #     "ohe_material_ALC（軽量気泡コンクリート）",
    #     "ohe_material_HPC（プレキャスト・コンクリート（重量鉄骨））",
    #     "ohe_material_PC（プレキャスト・コンクリート（鉄筋コンクリート））",
    "ohe_material_RC（鉄筋コンクリート）",
    "ohe_material_SRC（鉄骨鉄筋コンクリート）",
    "ohe_material_その他",
    #     "ohe_material_ブロック",
    "ohe_material_木造",
    "ohe_material_軽量鉄骨",
    "ohe_material_鉄骨造",
    "area",
    "contract",
    "age_year",
    "is_bus_1",
    "duration_1_1min",
    "duration_1_3min",
    "duration_1_5min",
    "duration_1_7min",
    "duration_1_10min",
    "duration_1_15min",
    "duration_1_20min",
    "duration_1_more_than_20min",
]

In [130]:
pd.set_option("display.max_rows", 999)
pd.DataFrame(
    {
        "col_name": pd.Series(train_df.drop(["target"], axis=1).columns).map(
            rename_cols
        ).fillna(pd.Series(train_df.drop(["target"], axis=1).columns)),
        "fi": lgb_models[0].feature_importance(),
    }
).sort_values("fi", ascending=False)

,col_name,fi
9,target_address_1_2_area,4941
1,target_address_1_2_mean,4792
111,age_year,4678
2,target_station_1_mean,4354
11,target_station_2_area,4214
10,target_station_1_area,4204
3,target_station_2_mean,4180
8,target_address_1_area,4156
5,target_devided_by_area_address_1_2_mean,3753
7,target_devided_by_area_station_2_mean,3517


In [125]:
from src.exp.common.setup import *
from src.exp.common.load_data import load_data
from src.exp.common.correct_invalid_data import correct_invalid_data
from src.exp.common.preprocessing import preprocessing
from src.exp.common.fill_missing_data import fill_missing_data
from src.exp.common.one_hot_encoding import one_hot_encoding
from src.exp.common.create_cv_df import create_cv_df
from src.exp.common.target_encoding import target_encoding
from src.exp.common.drop_unused_columns import drop_unused_columns
from src.exp.common.rename_non_ascii_cols import rename_non_ascii_cols

# Step 1. Load train.csv, test.csv
with timer("load data"):
    train_df, test_df = load_data(DATA_DIR=DATA_DIR)
    original_train_df, original_test_df = load_data(DATA_DIR=DATA_DIR)
    train_df = correct_invalid_data(train_df)
    original_train_df = correct_invalid_data(original_train_df)

# Step 2. Preprocessing
with timer("preprocessing"):
    train_df = preprocessing(df=train_df)
    test_df = preprocessing(df=test_df)

    # fill missing data
    for col_name in ["direction", "material", "layout"]:
        train_df = fill_missing_data(
            df=train_df, col_name=col_name, method="most_frequent"
        )
        test_df = fill_missing_data(
            df=test_df, col_name=col_name, method="most_frequent"
        )

train_df["target_devided_by_area"] = train_df["target"] / train_df["area"]

# Step 3. Feature Engineering

# Step 4. One-Hot Encoding
with timer("one hot encoding"):
    for col_name in ["direction", "layout", "material"]:
        train_df = one_hot_encoding(df=train_df, col_name=col_name)
        test_df = one_hot_encoding(df=test_df, col_name=col_name)

# Step 5. Create dataframe for CV
with timer("create dataframe for CV"):
    cv_df = create_cv_df(n_splits=N_FOLDS, random_state=SEED, train_df=train_df)

# Step 6. Target Encoding
with timer("target encoding"):
    col_names = ["address_1", "address_1_2", "station_1", "station_2"]
    target_col_names = ["target", "target_devided_by_area"]
    methods = {"mean": np.mean}

    cv_df_tmp = cv_df.copy()
    cv_df = pd.DataFrame()
    for n_fold in cv_df_tmp["n_fold"].unique():
        tr_df = cv_df_tmp.query(
            "n_fold == {} and data_type == '{}'".format(n_fold, "train")
        )
        te_df = cv_df_tmp.query(
            "n_fold == {} and data_type == '{}'".format(n_fold, "val")
        )
        for col_name in col_names:
            for target_col_name in target_col_names:
                tr_df, te_df = target_encoding(
                    tr_df=tr_df,
                    te_df=te_df,
                    col_name=col_name,
                    target_col_name=target_col_name,
                    methods=methods,
                )
        cv_df = pd.concat([cv_df, tr_df, te_df])

    for col_name in col_names:
        for target_col_name in target_col_names:
            train_df, test_df = target_encoding(
                tr_df=train_df, te_df=test_df, col_name=col_name, target_col_name=target_col_name, methods=methods
            )

    def fill_target_nan(df=pd.DataFrame()):
        for target_col_name in target_col_names:
            df[target_col_name + "_address_1_2_mean"] = df[target_col_name + "_address_1_2_mean"].fillna(
                df[target_col_name + "_address_1_mean"]
            )
            df[target_col_name + "_station_1_mean"] = df[target_col_name + "_station_1_mean"].fillna(
                df[target_col_name + "_address_1_mean"]
            )
            df[target_col_name + "_station_2_mean"] = df[target_col_name + "_station_2_mean"].fillna(
                df[target_col_name + "_address_1_mean"]
            )
        #         df["target_building_mean"] = df["target_building_mean"].fillna(df["target_address_1_2_mean"])
        #         df["target_building_mean"] = df["target_building_mean"].fillna(df["target_address_1_mean"])
        return df

    cv_df = fill_target_nan(cv_df)
    train_df = fill_target_nan(train_df)
    test_df = fill_target_nan(test_df)
    
for col_name in ["address_1", "address_1_2", "station_1", "station_2"]:
    train_df["target_" + col_name + "_area"] = train_df["target_devided_by_area_" + col_name + "_mean"] * train_df["area"]
    test_df["target_" + col_name + "_area"] = test_df["target_devided_by_area_" + col_name + "_mean"] * test_df["area"]
    cv_df["target_" + col_name + "_area"] = cv_df["target_devided_by_area_" + col_name + "_mean"] * cv_df["area"]

# Step 7. Drop unused columns
with timer("drop unused columns"):
    train_df = train_df[use_cols]
    test_df = test_df[use_cols[1:]]
    cv_df = cv_df[["index"] + use_cols + ["data_type", "n_fold"]]

# Step 8. Rename non-ascii columns as lightGBM doesn't support them.
with timer("rename non-ascii cols"):
    train_df, _ = rename_non_ascii_cols(df=train_df)
    test_df, _ = rename_non_ascii_cols(df=test_df)
    cv_df, rename_cols = rename_non_ascii_cols(df=cv_df)

2019-11-06 12:52:53,495 - INFO - [load data] done in 2.67 s
2019-11-06 12:53:12,910 - INFO - [preprocessing] done in 19.41 s
2019-11-06 12:53:13,325 - INFO - [one hot encoding] done in 0.41 s
2019-11-06 12:53:20,152 - INFO - [create dataframe for CV] done in 6.82 s
2019-11-06 12:53:26,668 - INFO - [target encoding] done in 6.51 s
2019-11-06 12:53:31,436 - INFO - [drop unused columns] done in 4.75 s
2019-11-06 12:53:31,555 - INFO - [rename non-ascii cols] done in 0.11 s


In [126]:
# LightGBM parameters
lgb_params = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": {"l2"},
    "num_leaves": 5000,
    "learning_rate": 0.1,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 20,
    "verbose": -1,
}

# xgb parameters
xgb_params = {
    "eta": 0.1,
    "max_depth": 10,
    "gamma": 0.1,
    "objective": "reg:squarederror", 
    "eval_metric": "rmse"
}

In [127]:
# Step 5. Cross Validation
from src.exp.common.cv import cv

with timer("cv with linear regression"):
    lr_result, scores, lr_models = cv(
        cv_df=cv_df, model_name="linearRegression", params=None
    )

    # xgb parameters
    xgb_params = {
        "eta": 0.1,
        "max_depth": 10,
        "gamma": 0.1,
        "objective": "reg:squarederror", 
        "eval_metric": "rmse"
    }
    xgb_result, scores, xgb_models = cv(
        cv_df=cv_df, model_name="xgboost", params=xgb_params
    )
    
    lgb_result, scores, lgb_models = cv(
        cv_df=cv_df, model_name="lightgbm", params=lgb_params
    )
    

---------------- linearRegression ----------------
n_fold: 0 Score: 23696.73299833869
n_fold: 1 Score: 30985.555867988216
n_fold: 2 Score: 32289.20512892736
n_fold: 3 Score: 36837.020606230304
n_fold: 4 Score: 24669.09864170425
---------------- linearRegression  END ----------------

---------------- xgboost ----------------
n_fold: 0 Score: 15487.724148983303
n_fold: 1 Score: 14294.407929132904
n_fold: 2 Score: 14623.619502084159
n_fold: 3 Score: 21595.48089068541
n_fold: 4 Score: 15413.056003117483
---------------- xgboost  END ----------------

---------------- lightgbm ----------------
n_fold: 0 Score: 16635.143884763776
n_fold: 1 Score: 15007.983840289946
n_fold: 2 Score: 16084.376144417285
n_fold: 3 Score: 19628.629281932448
n_fold: 4 Score: 17425.633794812293
---------------- lightgbm  END ----------------



2019-11-06 12:54:31,254 - INFO - [cv with linear regression] done in 59.37 s


In [128]:
from src.exp.common.plot_result import plot_result


@interact(n_fold=list(range(N_FOLDS)))
def plot_results(n_fold):
    plot_result(original_train_df, lr_result, n_fold, "lr")
    plot_result(original_train_df, xgb_result, n_fold, "xgb")
    plot_result(original_train_df, lgb_result, n_fold, "lgb")

interactive(children=(Dropdown(description='n_fold', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_classe…

In [131]:
# train and predict
from src.exp.common.train_and_predict import train_and_predict

y_pred, model = train_and_predict( 
    train_x=train_df.drop(["target"], axis=1),
    train_y=train_df["target"].values,
    val_x=test_df,
    model_name="xgboost",
    params=xgb_params,
)

result_df = pd.DataFrame({
    "id": test_df.reset_index()["index"]+31471,
    "predicted": y_pred
})
result_df.to_csv(
    "submission_{}_{}.csv".format(pd.to_datetime("today").strftime("%Y-%m-%d"), 1),
    header=False,
    index=False,
)

In [135]:
pd.concat([
    pd.DataFrame({
        "predicted": y_pred
    }),
    test_df
], axis=1)

,predicted,target_address_1_mean,target_address_1_2_mean,target_station_1_mean,target_station_2_mean,target_devided_by_area_address_1_mean,target_devided_by_area_address_1_2_mean,target_devided_by_area_station_1_mean,target_devided_by_area_station_2_mean,target_address_1_area,...,age_year,is_bus_1,duration_1_1min,duration_1_3min,duration_1_5min,duration_1_7min,duration_1_10min,duration_1_15min,duration_1_20min,duration_1_more_than_20min
0,121249.554688,113781.270261,161526.315789,125041.666667,124856.287425,3423.470721,3088.343425,2977.634232,3402.643714,171926.699614,...,49,0,0,0,0,0,0,1,0,0
1,115081.265625,145674.389052,143592.307692,151232.222222,144863.559322,4075.624481,4156.924899,4022.850983,3856.480379,85099.039161,...,0,0,0,0,0,1,0,0,0,0
2,77665.648438,110916.884343,110013.953488,103259.183673,105198.601399,3894.012945,3981.513629,3558.966407,3670.860869,104865.768618,...,23,0,0,0,0,0,1,0,0,0
3,59269.054688,97708.174675,88017.441860,97172.303704,99133.561644,3336.672242,3577.162432,3557.014864,3957.313806,78645.364745,...,36,0,0,0,0,0,1,0,0,0
4,117877.179688,97708.174675,92972.482353,97172.303704,88614.400000,3336.672242,3413.753185,3557.014864,3659.635304,166833.612103,...,45,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31257,101091.812500,110916.884343,120390.217391,101448.148148,105057.377049,3894.012945,3899.552989,3546.065943,3488.178906,99920.372178,...,0,0,0,1,0,0,0,0,0,0
31258,97885.125000,182706.236324,130960.000000,138679.245283,150945.054945,4755.474887,4277.503890,4640.391068,4492.477982,107996.834692,...,15,0,0,1,0,0,0,0,0,0
31259,170204.531250,101578.398018,162976.635514,150815.573770,143924.812030,3629.443669,3471.978537,3549.847542,3429.135869,166083.342300,...,1,0,0,0,0,0,1,0,0,0
31260,269979.562500,182706.236324,385307.692308,405000.000000,307173.076923,4755.474887,5093.786693,4874.933492,4507.530339,262502.213783,...,15,0,0,1,0,0,0,0,0,0
